In [8]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests

dbname = "fdac18mp2" #please use this database
collname = "glprj_lpassare" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]
letter="b"

beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"


gleft = 10

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

# send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if el['name'].lower().startswith(letter):
                    el['site'] = "git"
                    count += 1
                    coll.insert_one(el)
                    if count > 49:
                        return
            
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(letter):
                                el['site'] = "git"
                                count += 1
                                coll.insert_one(el)
                                if count > 49:
                                    return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
#start retrieving        
get(beginurl,coll)

https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=0&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false;localhost:27017: [Errno 111] Connection refused


### Source forge

In [14]:

page_number= 20


url=[]
for n in range(1,page_number):
    url.append("https://sourceforge.net/directory/os%3Amac/?q=j&page=" + str(n))
    
urls = [requests.get(link) for link in url]
from bs4 import BeautifulSoup
project_names=[]
for resp in urls:
    html_soup= BeautifulSoup(resp.text, 'html.parser')
    soup_string= str(html_soup)
    regex = "(?<=\/projects\/).+?(?=\/)"
    matches = re.findall(regex, soup_string)
    b_start= [i for i in matches if i.startswith('b')]
    project_names.append(b_start)

In [15]:
print(project_names)

[[], [], ['bemoda', 'bemoda', 'bemoda', 'bemoda'], ['bayesrate', 'bayesrate', 'bayesrate', 'bayesrate', 'bayesrate', 'beat-em-up-game', 'beat-em-up-game', 'beat-em-up-game', 'beat-em-up-game'], ['barcode4j', 'barcode4j', 'barcode4j', 'barcode4j', 'barcode4j'], ['bayesianestimatedcoregenome', 'bayesianestimatedcoregenome', 'bayesianestimatedcoregenome', 'bayesianestimatedcoregenome', 'bmnews', 'bmnews', 'bmnews', 'bmnews'], [], [], ['bc-j-mapi-w', 'bc-j-mapi-w', 'bc-j-mapi-w', 'bc-j-mapi-w'], ['beesalgorithm', 'beesalgorithm', 'beesalgorithm', 'beesalgorithm'], ['byaccj', 'byaccj', 'byaccj', 'byaccj'], ['buffalo', 'buffalo', 'buffalo', 'buffalo'], [], [], ['businessj', 'businessj', 'businessj', 'businessj'], ['bldc', 'bldc', 'bldc', 'bldc'], [], [], []]
